In [ ]:


import cv2
import dlib
import imutils
from imutils import face_utils
import numpy as np
from scipy.spatial import distance as dist
import time
import winsound
import base64
from PIL import Image
import jwt
import os
import socketio
import io
import requests
import random

from flask import Flask,flash, render_template, request, redirect, url_for, jsonify, session

Url= 'https://peach-violet-rhinoceros.glitch.me/api/v1/'
studentUrl=Url+"student/login"


app = Flask(__name__, template_folder='templates', static_folder='static')

app.secret_key = 'session'


sio = socketio.Client()
sio.connect("http://localhost:5000/")


camera = cv2.VideoCapture(0)



@app.route('/', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password'] 
        data = {'emailID': username, 'password': password}
        response = requests.post(studentUrl, json=data)
        if response.json()['success']:
            # If the authentication was successful, store the token in the user's session
            token = response.json()['data']['tokem']
            session['user_data'] = {
                'token': token,
                'student': {
                    '_id': response.json()['data']['student']['_id'],
                    'emailID': response.json()['data']['student']['emailID'],
                    'name': response.json()['data']['student']['firstName'],
                    'emailVerified': response.json()['data']['student']['emailVerified'] ,
                }
            }
            print("User Logged in Successfully")
            return redirect(url_for('join_meeting'))
        else:
            message = 'Invalid Details!!'
            flash(message)     
    else:
        return render_template('login.html')
             

@app.route('/meeting_room', methods=['GET', 'POST'])
def meeting_room():
    sio.emit('join_room', {'_id': session['room_id'], 'student': session['user_data']['student']})
    return render_template('meeting_room.html')
    
    

@app.route('/join_meeting', methods=['GET','POST'])
def join_meeting():
    print("Hi")
    if "user_data" in session:
        print("Hello LoggedIn User!")
        if request.method == 'POST':
            session_id=request.form['room']
            joinSessionUrl=Url+"room/"+session_id
            print(joinSessionUrl)
            response=requests.patch(joinSessionUrl,  headers={
                   'Authorization': session['user_data']['token']})
            if response.json()['success']:
                print("Authorization Completed")
                
                session['room_id'] = session_id

                return redirect(url_for('meeting_room'))
            else:
                print("Invalid Details try again")
                return render_template('join_meeting.html')
            
            
        else:
            return render_template('join_meeting.html')
           

    else:
        print("Please login first");
        return render_template('login.html')
        
def get_state_value(detection):
    if detection=="drowsy":
        return random.randint(10, 33)
    elif detection=="unfocused":
        return random.randint(34, 59)
    
    else:
        return random.randint(60, 95)
        

@app.route('/run_eye_tracking')
def run_eye_tracking():
    def play_alarm():
    #time.sleep(3)
        frequency = 3500  # Set Frequency To 2500 Hertz
        duration = 5000  # Set Duration To 1000 ms == 1 second
        winsound.Beep(frequency, duration)
   

# Calculate the eye aspect ratio
    def eye_aspect_ratio(eye):
        # Compute the euclidean distances between the two sets of vertical eye landmarks (x, y)-coordinates
        A = dist.euclidean(eye[1], eye[5])
        B = dist.euclidean(eye[2], eye[4])

        # Compute the euclidean distance between the horizontal eye landmark (x, y)-coordinates
        C = dist.euclidean(eye[0], eye[3])

        # Compute the eye aspect ratio
        ear = (A + B) / (2.0 * C)

        # Return the eye aspect ratio
        return ear

    # Initialize dlib's face detector and create a dictionary to map facial landmark indexes to face parts
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    face_parts = face_utils.FACIAL_LANDMARKS_IDXS
    (left_eye_start, left_eye_end) = face_parts["left_eye"]
    (right_eye_start, right_eye_end) = face_parts["right_eye"]

    # Start the video stream and wait for the camera to warm up
    vs = cv2.VideoCapture(0)
    time.sleep(2.0)

    # Initialize variables for calculating the attention level
    drowsy_counter = 0
    drowsy_threshold = 15*6
    focused_counter = 0
    focused_threshold = 5*6
    unfocused_counter=0;
    head_direction_threshold = 5*6
    head_direction_degrees = 0
    max_attention_level = "unfocused"
    max_attention_time = time.time()

    # Initialize blink detector
    blink_frames = 0
    blink_threshold = 3
    eyes_closed = False

    # Loop over frames from the video stream
    while True:
        # Read the next frame from the video stream and resize it
        _, frame = vs.read()
        frame = imutils.resize(frame, width=450)

        # Convert the frame to grayscale and detect faces
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        # Loop over the face detections
        for rect in rects:
            # Determine the facial landmarks for the face region, then convert the facial landmark (x, y)-coordinates to a NumPy array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # Extract the left and right eye coordinates and compute the eye aspect ratio for each eye
            left_eye = shape[left_eye_start:left_eye_end]
            right_eye = shape[right_eye_start:right_eye_end]
            left_eye_ear = eye_aspect_ratio(left_eye)
            right_eye_ear = eye_aspect_ratio(right_eye)

            # Compute the average eye aspect ratio
            avg_ear = (left_eye_ear + right_eye_ear) / 2.0
            
            
            # Check if the head is directed towards the screen
            # Calculate the angle between the eye line and the horizontal line
            left_eye_center = np.mean(left_eye, axis=0).astype(int)
            right_eye_center = np.mean(right_eye, axis=0).astype(int)
            dx = right_eye_center[0] - left_eye_center[0]
            dy = right_eye_center[1] - left_eye_center[1]
            angle = np.degrees(np.arctan2(dy, dx))
            if angle < 0:
                angle += 360
            head_direction_degrees = angle
    #         print(head_direction_degrees)
            if head_direction_degrees > 185 or head_direction_degrees < 170:
                unfocused_counter+=1
                drowsy_counter = 0
                focused_counter=0
            else:
                unfocused_counter=0

            # Check if the average eye aspect ratio is below the drowsy threshold
            if avg_ear < 0.25:
                drowsy_counter += 1
                focused_counter = 0
            else:
                drowsy_counter = 0
                focused_counter += 1

            # Check if the attention level is drowsy or unfocused for more than 3 seconds
            if drowsy_counter >= drowsy_threshold:
                max_attention_level = "drowsy"
#                 print(drowsy_counter)
                if drowsy_counter == drowsy_threshold:
                    alarm_start_time = time.time()
                if time.time() - alarm_start_time >= 3 and not eyes_closed:
                    play_alarm()
                    eyes_closed = True
                    drowsy_counter = 0
            elif focused_counter >= focused_threshold:
                max_attention_level = "focused"
#                 print(focused_counter)
                eyes_closed = False
                
                
            elif unfocused_counter >= head_direction_threshold:
                    max_attention_level = "unfocused"
#                     print(unfocused_counter)
                    unfocused_counter=0
                    eyes_closed = False
        
    # Update the max attention level
        if time.time() - max_attention_time >= 10:
            sio.emit('update_attention', {'_id': session['room_id'], 'student': session['user_data']['student'], 'attetnion': get_state_value(max_attention_level ), 'detection': max_attention_level})
#             print(f"Max attention level for the past 10 Seconds : {max_attention_level} - {get_state_value(max_attention_level )}")
            max_attention_level = "unfocused"
            if time.time() - max_attention_time >= 60:
                print(max_attention_level)
                max_attention_time = time.time()
        
     
            
    
        

        # Display the frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF

        # If the `q` key was pressed, break from the loop
        if key == ord("q"):
            break
        

    # Cleanup
    cv2.destroyAllWindows()
    #vs.stop()
    # return Response(generate_frames(),mimetype='multipart/x-mixed-replace; boundary=frame')
    


    

if __name__ == '__main__':
    app.run(debug=False, port=5005)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5005/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Apr/2023 13:30:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Apr/2023 13:30:11] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [20/Apr/2023 13:30:11] "GET /join_meeting HTTP/1.1" 200 -


User Logged in Successfully
Hi
Hello LoggedIn User!
Hi
Hello LoggedIn User!
https://peach-violet-rhinoceros.glitch.me/api/v1/room/643e7d09f2d4844eac0daefa


127.0.0.1 - - [20/Apr/2023 13:30:19] "POST /join_meeting HTTP/1.1" 302 -
127.0.0.1 - - [20/Apr/2023 13:30:19] "GET /meeting_room HTTP/1.1" 200 -


Authorization Completed
unfocused


[2023-04-20 13:32:06,101] ERROR in app: Exception on /run_eye_tracking [GET]
Traceback (most recent call last):
  File "E:\New folder\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "E:\New folder\lib\site-packages\flask\app.py", line 1953, in full_dispatch_request
    return self.finalize_request(rv)
  File "E:\New folder\lib\site-packages\flask\app.py", line 1968, in finalize_request
    response = self.make_response(rv)
  File "E:\New folder\lib\site-packages\flask\app.py", line 2097, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The function either returned None or ended without a return statement.
127.0.0.1 - - [20/Apr/2023 13:32:06] "GET /run_eye_tracking HTTP/1.1" 500 -
